# 06_Vertical Integrated Moisture Transport

In this chapter, we aim to diplay moisture transports over the globe. As we will see, we can plot the integrated moisture transport in a similar way as a windfield!

## 06_01 Background


The essence of the overall hydrological cycle is the evaporation of moisture ($E$) in one place and the precipitation in other places ($P$). In particular, evaporation exceeds precipitation over the oceans, which allows moisture to be transported by the atmosphere onto land where precipitation exceeds evapotranspiration, and the runoff flows into streams and rivers and discharges into the ocean, completing the cycle (Trenberth et al., 2011).


This cycle can be expressed in a equation. The atmospheric conservation of moisture equation when vertically integrated in flux form is

$$\frac{\partial w}{\partial t}+ \nabla\cdot\frac{1}{g}\int_{0}^{p_s} vq  dp = E - P$$ 

Where $q$ is the specific humidity, $v$ the horizontal wind vector, $w = g^{-1}\int_{0}^{p_s}qdp$ is the precipitable water (total column water vapor), $E$ is the surface evaporation, and $P$ is the net surface precipitation rate.

The second term on the left hand side of the equation is the divergence of the vertical integrated moisture flux. 
Areas for which there is an excess of evaporation over precipitation ($E − P > 0$) are moisture source regions over which the atmospheric moisture fluxes diverge (divergence term >0), and other areas, where moisture fluxes converge, are moisture sink regions.

The local change of the total column water vapor is therefore dependent on the local precipitation, evaporation and the moisture flux which transports moisture from neigbouring columns into the respective column.

To illustrate, how moisture is tranported on our globe and where it origins, we can either plot the integrated moisture flux or its divergence. The divergence will show us the sink and source regions of moisture as explained above.Let's have a quick look at the integrated moisture flux itself:

$\frac{1}{g}\int_{0}^{p_s} vq  dp$

A flux generally consists of the transported quantity, in this case the specific humidity $q$ in [kg/kg] and a velocity, in this case $v$. Integrated from the TOA @ 0 Pa to the surface @ $p_s$ and multplied with $g$, we get the unit $Pa \cdot m s^{-1}\cdot s^2 m^{-1}$, which gives $kg m^{-1}s^{-1}$ on the whole. 

## 06_02 The Data

Let's load the data of vertical integrated moisture fluxes and have a closer look at it:

In [ ]:
# Display the plots in the notebook:
%matplotlib inline

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are to ignore certain warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds = xr.open_dataset('./data/ERA5-LowRes-MonthlyAvg-VertIntegratedMoistureFlux.nc')

In [ ]:
ds = ds.rename({"p71.162_0001" : "EMT", "p72.162_0001":"NMT"})

In [ ]:
ds.EMT

As expected, the unit of the integrated water vapour flux is $kg m^{-1}s^{-1}$! 
We have a variable for the eastward transport and one for the northward transport of water vapour with $u$, the zonal wind or $v$, the meridional wind respectively. This is pretty similar to having $u$ and $v$ as the vector components of the horizontal windfield. Hence, we can take the eastward and northward vertical integral of water vapour flux as the vector components of the integrated water vapour flux field! 

We can plot this field as a vectorfield; this is shown below.

## 06_03 How to Plot it

### 06_03_01 Vertical Integral of  Water Vapour Flux

Similar to when we plotted the windfield, we start by averaging over the months of the year and selecting a specific month (january).

In [ ]:
emt = ds.EMT.groupby('time.month').mean(dim = 'time').sel(month=1).load()
nmt = ds.NMT.groupby('time.month').mean(dim = 'time').sel(month=1).load()

Now plot the vectorfield with `quiver`:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
e, n = emt[::10,::10], nmt[::10,::10] #read 05_PlotWindData for explanations on this!
qv = ax.quiver(e.longitude, e.latitude, e, n, transform=ccrs.PlateCarree())
ax.coastlines(color='grey');
ax.set_title('Average Integrated Water Vapour Flux January [$kg m^{-1} s^{-1}$]');

As expected and mentioned above, the oceans are a source of water vapour and therfore a region where the water vapour flux diverges!

Again, we lack a discriptive information about the values of the depicted arrows. We can plot this as a shaded colour plot in the background.

Therefore, we must first determine the absolute values of the vectors:

In [ ]:
av = np.sqrt(emt**2+nmt**2)
av

And now plot it:

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())  
av.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels = 10, cmap='cool', cbar_kwargs={'label':'Water Vapour Flux [$kg m^{-1} s^{-1}$]'});
e, n = emt[::10,::10], nmt[::10,::10]
qv = ax.quiver(e.longitude, e.latitude, e, n, transform=ccrs.PlateCarree())
ax.coastlines(color='grey');
ax.set_title('Average Integrated Water Vapour Flux January');

We can also plot one of the vector components of the flux field in a 1D-plot! E.g. the northward water vapour flux, which is proportional to the energy transported by the latent heat flux. Values greater than zero indicate a northward transport, values smaller than zero a southward transport of water vapour and therefore latent heat.

In [ ]:
nmt_zonal = ds.NMT.mean(dim = ['time', 'longitude'])
nmt_zonal.plot()
plt.xlim(-90, 90) # defines the limits on the x-axis
plt.title('Vertical integral of northward water vapour flux') # plots your title
plt.ylabel('[kg $m^{-1}s^{-1}$]'); # changes the y-label from the default label to the one you want
plt.xlabel('Latitude [° North]');
plt.xticks((-90, -60, -30, 0, 30, 60, 90));
plt.axhline(0, color = 'k');
plt.grid(linestyle = '-.')

### 06_03_02 Vertical integral of divergence of water vapour flux

We can also plot the divergence of the flux we plotted above. Let's load the data first:

In [ ]:
ds = xr.open_dataset('./data/ERA5-LowRes-MonthlyAvg-VertIntDivMoistFlux.nc')
ds = ds.rename({"p84.162" : "div"})
ds.div

Since it is a divergence of the moisture flux field, the unit changes from $kg m^{-1}s^{-1}$ to $kg m^{-2}s^{-1}$. We can transform this unit into $mm/day$ by multiplying it with $24\cdot60\cdot60$, the seconds of a day! This is a more intuitiv unit.

Let's do this and then plot it:

In [ ]:
div_jan = ds.div.groupby('time.month').mean(dim = 'time').sel(month = 1)
div_jan = div_jan *24*60*60
ax = plt.axes(projection=ccrs.Robinson())
div_jan.plot(ax=ax, transform=ccrs.PlateCarree(), levels = np.arange(-8, 8, 2),vmin = -8, vmax= 8, cmap = 'cool', 
             cbar_kwargs={'label': '$mm/day$'}) 
ax.set_title('Averaged Vertical integral of divergence of moisture flux January')
ax.coastlines(); ax.gridlines();

The major source regions for atmospheric moisture can be identified in the subtropical anticyclones, and major sink regions in the convergence zones and summer monsoon rains (Trenberth et al., 2011). Trenberth furhtermore states, that the tendency term $\frac{\partial w}{\partial t}$ is very small for time averages. Therefore, one can take the divergence of the moisture flux as proportional to $E-P$. 